In [1]:
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install backtesting

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import os
import pandas as pd
#from strategy_utils import *
import time
from os.path import exists
import pandas_ta as ta
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting import Backtest
#import mysql.connector
#from mysql.connector import Error

/usr/local/lib/python3.7/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [5]:
from google.colab import files
files.upload()

Saving HINDALCO_1D.xlsx - HINDALCO.csv to HINDALCO_1D.xlsx - HINDALCO (1).csv


{'HINDALCO_1D.xlsx - HINDALCO.csv': b'datetime,close,high,low,open,volume,instrument\r\n2014-01-24 00:00:00,114,115.35,113,113.15,5737135,HINDALCO\r\n2014-01-27 00:00:00,111.1,112.7,109.3,112,8724577,HINDALCO\r\n2014-01-28 00:00:00,113.8,115,109.75,110,4513345,HINDALCO\r\n2014-01-29 00:00:00,111.75,114.75,111.15,114.5,4713458,HINDALCO\r\n2014-01-30 00:00:00,108.1,110.7,107.6,110.2,5077231,HINDALCO\r\n2014-01-31 00:00:00,109.55,110,107,109.05,8287236,HINDALCO\r\n2014-02-03 00:00:00,103.45,109.4,103,109,5572627,HINDALCO\r\n2014-02-04 00:00:00,102.45,103.45,99.8,102,13935173,HINDALCO\r\n2014-02-05 00:00:00,104.25,105.15,101,102.4,9106760,HINDALCO\r\n2014-02-06 00:00:00,105.75,106.5,103.1,104.65,10194884,HINDALCO\r\n2014-02-07 00:00:00,106.4,107.7,104.75,106.15,9650511,HINDALCO\r\n2014-02-10 00:00:00,106.8,107.7,105.8,106.45,3124538,HINDALCO\r\n2014-02-11 00:00:00,104.55,107.1,104.25,106.9,3964390,HINDALCO\r\n2014-02-12 00:00:00,103.5,105.6,103.05,105.15,5325176,HINDALCO\r\n2014-02-13 00:0

In [6]:
# install
!apt install postgresql postgresql-contrib &>log

In [8]:
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root1 WITH SUPERUSER"

 * Starting PostgreSQL 10 database server
   ...done.
CREATE ROLE


In [9]:
# set connection
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


'Connected: @postgres'

In [10]:
# create table before load
%%sql
CREATE TABLE security
(
  id   serial PRIMARY KEY,
  datetime date,
	open float ,
  high float ,
  low float ,
  close float,
  volume int,
  instrument VARCHAR(30),
)
# CSV to load
%%file HINDALCO_1D.xlsx - HINDALCO.csv
datetime,open,high,low,close,volume,instrument
# Load
%%sql
COPY HINDALCO_1D.xlsx - HINDALCO(datetime,open,high,low,close,volume,instrument)
FROM '/content/HINDALCO_1D.xlsx - HINDALCO.csv' DELIMITER ',' CSV HEADER;
# Check result
%sql SELECT * FROM HINDALCO_1D.xlsx - HINDALCO.csv

 * postgresql+psycopg2://@/postgres
(psycopg2.ProgrammingError) syntax error at or near ")"
LINE 11: )
         ^

[SQL: CREATE TABLE security
(
  id   serial PRIMARY KEY,
  datetime date,
	open float ,
  high float ,
  low float ,
  close float,
  volume int,
  instrument VARCHAR(30),
)
# CSV to load
%%%%file HINDALCO_1D.xlsx - HINDALCO.csv
datetime,open,high,low,close,volume,instrument
# Load
%%%%sql
COPY HINDALCO_1D.xlsx - HINDALCO(datetime,open,high,low,close,volume,instrument)
FROM '/content/HINDALCO_1D.xlsx - HINDALCO.csv' DELIMITER ',' CSV HEADER;]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [11]:
# dump table persons to file.sql
!pg_dump -d postgres -t HINDALCO_1D.xlsx - HINDALCO --no-owner --inserts > HINDALCO_1D.xlsx - HINDALCO.sql
# view it
!cat HINDALCO_1D.xlsx - HINDALCO.sql | sed -n -e 24,35p -e 64,70p
# load it back
!psql postgres -c "DROP TABLE HINDALCO_1D.xlsx - HINDALCO"
!psql postgres < HINDALCO_1D.xlsx - HINDALCO.sql

pg_dump: too many command-line arguments (first is "-")
Try "pg_dump --help" for more information.
pg_dump --help
open
-
^C
ERROR:  syntax error at or near "-"
LINE 1: DROP TABLE HINDALCO_1D.xlsx - HINDALCO
                                    ^
psql: warning: extra command-line argument "HINDALCO.sql" ignored
psql: FATAL:  Peer authentication failed for user "-"


In [12]:
df = pd.read_csv("HINDALCO_1D.xlsx - HINDALCO.csv")
df

,datetime,close,high,low,open,volume,instrument
0,2014-01-24 00:00:00,114.00,115.35,113.00,113.15,5737135,HINDALCO
1,2014-01-27 00:00:00,111.10,112.70,109.30,112.00,8724577,HINDALCO
2,2014-01-28 00:00:00,113.80,115.00,109.75,110.00,4513345,HINDALCO
3,2014-01-29 00:00:00,111.75,114.75,111.15,114.50,4713458,HINDALCO
4,2014-01-30 00:00:00,108.10,110.70,107.60,110.20,5077231,HINDALCO
...,...,...,...,...,...,...,...
1210,2018-12-21 00:00:00,224.00,227.45,221.80,224.25,6933691,HINDALCO
1211,2018-12-24 00:00:00,217.85,224.00,216.95,224.00,4667022,HINDALCO
1212,2018-12-26 00:00:00,218.60,219.80,212.90,213.85,6554865,HINDALCO
1213,2018-12-27 00:00:00,220.60,223.90,219.60,221.40,7947144,HINDALCO


In [13]:
df = df.rename(columns={"open": "Open", "instrument":"instrument","close": "Close", "low": "Low", "high": "High", "volume": "Volume"})


In [14]:
# test suite
import unittest

class TestType(unittest.TestCase):
	# test function to test whether obj is instance of class
  def setUp(self):
      self.number = df['Volume']
      self.obj = df['instrument']
      self.dec = df['Open'],df['High'],df['Low'],df['Close']
  def test_assert_is_with_type(self):
      self.assertTrue(type(self.number),int64)
  def test_assert_is_with_type(self):
      self.assertTrue(type(self.dec),float64)
  def test_assert_is_with_type(self):
      self.assertTrue(type(self.obj),object)


if __name__ == '__main__':
	unittest.main(argv=['first-arg-is-ignored'],exit=False)



.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


In [15]:
def EMA(df_in, window):
    df = df_in.copy()
    #ema_series = df['Close'].ewm(span=ema1_len, min_periods=ema1_len).mean()
    ema_series = ta.ema(df['Close'], length=window)
    return ema_series

def EMA2(df_in, window):
    df = df_in.copy()
    #ema_series2 = df['Close'].ewm(span=ema2_len, min_periods=ema2_len).mean()
    ema_series2 = ta.ema(df['Close'], length=window)
    return ema_series2

In [16]:
class EmaCrossoverStrategy(Strategy):
    ema1_len = 10
    ema2_len = 50
    def init(self):
        super().init()
        # Add indicators
        self.ema1 = self.I(EMA, self.data.df, self.ema1_len)
        self.ema2 = self.I(EMA2, self.data.df, self.ema2_len)
    def next(self):
        super().init()
        price = self.data.Close[-1]
        # Long entry
        if (not self.position) and crossover(self.ema1, self.ema2):
            #  Buy
            self.position.close()
            self.buy()
        # Long exit
        elif self.position and crossover(self.ema2, self.ema1):
            # Close any existing long trades, and sell the asset
            self.position.close()
            self.sell()

In [17]:
#symbol, #data_file_map
def run_backtest(df):
    df = df
    #data_interval = "1H"
    #  Get data file
    #key = f"{symbol}_{data_interval}"
    #df = data_file_map[key]
    #if df is None or len(df) == 0:
        #print(f"Empty file: {symbol}, interval: {data_interval}")
    # If exclusive orders (each new order auto-closes previous orders/position),
    # cancel all non-contingent orders and close all open trades beforehand
    bt = Backtest(df, EmaCrossoverStrategy, cash=10000, commission=.00075, trade_on_close=True, exclusive_orders=True, hedging=False)
    stats = bt.run()
    print(stats)
    bt.plot()

In [18]:
if __name__ == '__main__':
    procs = []
    #load_all_data_files(symbols, data_interval_array)
    run_backtest(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  if sys.path[0] == '':


Start                                     0.0
End                                    1214.0
Duration                               1214.0
Exposure Time [%]                   35.226337
Equity Final [$]                          0.0
Equity Peak [$]                   18296.47935
Return [%]                             -100.0
Buy & Hold Return [%]               96.052632
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                      -100.0
Avg. Drawdown [%]                   -9.835678
Max. Drawdown Duration                  710.0
Avg. Drawdown Duration              58.235294
# Trades                                  5.0
Win Rate [%]                             60.0
Best Trade [%]                       43.78545
Worst Trade [%]                   -101.256274
Avg. Trade [%]                    